# Path Following Algorithm API
## 개요
- 하드웨어 플랫폼에 적용가능한 벡터필드 유도알고리즘을 파이썬 환경에서 개발한다.
- 개발한 벡터필드 유도알고리즘을 실제 하드웨어 플랫폼에 적용하여 테스트한다.

## Hardware Platform
- Crazyflie

## 주요기능
- 2차원 경로생성기능
- 2차원 충돌회피기능

### 파이썬 개발환경
```zsh
conda install numpy plotly matplotlib scipy pandas sympy -y
conda install -c conda nbformat
```

### 서브모듈 업데이트
```zsh
git clone git@github.com:SuwonLee-KMU/fmcl_vfg.git
cd fmcl_vfg
git submodule init
git submodule update --recursive
git submodule foreach git pull origin main
```

In [2]:
import numpy as np
import math
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import pyPOA.PlanarVehicle as pv
from pyPOA import POA
import pyPOA.POA.Obstacles as obs
import plotly.figure_factory as ff

Plotly_colors = px.colors.qualitative.Plotly


Development Elements

1. Planar Bezier Curve / Planar PH Curve <- Hermite Interpolation
2. Space Bezier Curve
3. Vector Field-bases Guidance algorithm

In [8]:

# 곡선을 결정함
Ps = [
    [0, 0], # Pi
    [1, 0.3], # Pf
]
Ts = [
    [1, 0], # Ti
    [1,  0], # Tf
]
Ts = [POA.normalize(T) for T in Ts]
# Hermite Interpolation
h = POA.Curves.HermiteInterpolation(
    Ps = Ps,
    Ts = Ts,
    scaling = 0.2,
    mode = 0,
    tau = np.linspace(0,1,100)
)
# Planar PH Curve evaluation
q = POA.Curves.PlanarPHCurve(h)
tau = np.linspace(0,1,100)
pos = q(tau)
x = np.real(pos)
y = np.imag(pos)

In [18]:

# 장애물 객체 생성
cobs = []
cobs.append(obs.CircularObstacle(center=[0.3,0.1], radius=0.1))
cobs.append(obs.CircularObstacle(center=[0.7,0.2], radius=0.1))

# 샘플포인트 객체 생성 
num_sample_points = 100
phi = POA.ExponentialRBF(epsilon=num_sample_points) # 샘플포인트의 갯수와 같은 크기의 입실론을 사용하는 것이 좋다.(경험적)
spt = POA.OffsetCurves.SamplePoints(points=np.linspace(0,1,num_sample_points))
dlbc = POA.OffsetCurves.DLBComputer_multipleObstacle(Reference=q, SamplePointsObj=spt, step_length=0.01)
dlbc.stepThreshold = 10000
spt.DLB = dlbc.determine_DLBs(cobs)

# 오프셋곡선 객체 생성
oc = POA.OffsetCurves.OffsetCurve(Reference=q, SamplePoints=spt, RBF=phi)

# 곡선 위치 계산
pos_ref = q(tau,form="real")
pos_off = oc(tau)
vel_off = oc.derivative(tau)
pos_sp = q(spt.points, form="real")
index_collision = dlbc.get_obstacle_index(cobs)
pos_col = q(spt.points[index_collision], form="real")


RBFsum is ready for the OffsetCurve object.


Vector Field-based Guidance
- PlanarVehicle 모듈을 활용한다.
- 학생에게 시뮬레이션 하는 부분에 대한 설명을 진행한다.

In [19]:
class RefCurve(POA.Curves.PlanarPHCurve):
    """VFG를 적용하기 위해 __call__()메서드 오버라이딩"""
    def __call__(self, tau, **kwargs):
        return super().__call__(tau, form="real")
        

In [22]:

# 각 객체 생성
ref = RefCurve(h)                               # 기준곡선 객체
vstates = pv.States()                           # 상태변수 객체
control = pv.Inputs()                           # 제어입력 객체
guidance = pv.GNC.VectorFieldGuidance(
    ref, 20, gain=0.4, States=vstates)          # 유도기법 객체
my_vehicle = pv.Vehicle(InitialState=vstates)   # 비행체 객체
my_sensor = pv.Sensors.Proximity(
    radius=0.1, States=vstates, obs_list=cobs)  # 근접센서 객체
dtmaker = pv.GNC.DetourMaker(
    RefCurveObj=ref, num_sample_points=30)      # 우회로 생성기 객체
simout = pv.Simout()                            # 시뮬레이션 상태변수 로그 객체
simout_vfg = pv.Simout_VFG()                    # 시뮬레이션 유도기법 로그 객체
simout_sen = pv.Simout_Proximity(
    obs_list=cobs)                              # 시뮬레이션 근접센서 로그 객체

# 객체 및 변수 초기화
vstates.set_states([0, 0.01, 0, 0.1])           # 상태변수 초기화
my_sensor.update_sensing_list()                 # 센서 초기화
simout.add_states(my_vehicle.states, control)   # 시뮬레이션 상태변수 로그 시작
simout_vfg.add_states(0,0)                      # 시뮬레이션 유도기법 로그 시작
simout_sen.add_states(my_sensor.sensing_list)   # 시뮬레이션 근접센서 로그 시작
times = np.linspace(0,15, 401)                  # 시뮬레이션 시간 할당
dt = times[1] - times[0]
for t in times:
    # update sensor
    my_sensor.update_sensing_list()
    # update trajectory to follow
    if my_sensor.is_updated:
        dtmaker.sensed_obs = [obs for sensed, obs in zip(my_sensor.sensing_list, cobs) if sensed]
        guidance.CurveObject = dtmaker.oc
    # update guidance command
    guidance.States = my_vehicle.states
    vel_cmd = guidance.vel_cmd
    # update control command
    control.set_inputs([0, guidance.latax_cmd])
    # update states
    dstates = my_vehicle.dynamics(control)
    states_prev = my_vehicle.states.get_states()
    states_next = states_prev + dt*dstates
    my_vehicle.states.set_states(states_next)
    # Log simulation data
    simout.add_states(my_vehicle.states, control)
    simout_vfg.add_states(*vel_cmd)
    simout_sen.add_states(my_sensor.sensing_list)


RBFsum is ready for the OffsetCurve object.


In [23]:
quiver_steps = 10
layout = go.Layout(width=800, height=350, margin=dict(t=0,l=0,r=0,b=0),yaxis=dict(scaleanchor="x", scaleratio=1), legend=dict(x=0.01, y=0.99))
fig = go.Figure(layout=layout)
fig1 = ff.create_quiver(
    simout.x[::quiver_steps], 
    simout.y[::quiver_steps], 
    np.array(simout_vfg.u[::quiver_steps])*np.array(simout.speed[::quiver_steps]), 
    np.array(simout_vfg.v[::quiver_steps])*np.array(simout.speed[::quiver_steps]),
    name="vel_cmd")
fig2 = ff.create_quiver(simout.x[::quiver_steps], simout.y[::quiver_steps], simout.u[::quiver_steps], simout.v[::quiver_steps], name="vel")
fig.add_traces(data=fig1.data)
fig.add_traces(data=fig2.data)
fig.add_scatter(x=pos_ref[0], y=pos_ref[1], mode="lines", name="Reference path")
fig.add_scatter(x=simout.x, y=simout.y, mode="lines", name="Trajectory", line=dict(dash="dash"))
for cob in cobs:
    cob.add_patch(fig)

fig.show()